In [1]:
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.special import expit
import scipy.stats as sts


## Проверка гипотез

### 1. Проверим гипотезу о независимости расстояния до ближайшей больницы и исхода шутинга (смертностью жертв) с помощью теста отношения правдоподобий

In [2]:
dataa=pd.read_csv('df1.csv') #считываем обработанные данные из файла

In [3]:
d=pd.DataFrame(data=dataa, columns={'BORO','VIC_AGE_GROUP','VIC_SEX','VIC_RACE', 'STATISTICAL_MURDER_FLAG', 'distance', 
                                   'PERP_AGE_GROUP', 
                                   'PERP_SEX', 'PERP_RACE' })
d = pd.get_dummies(d, columns=['BORO','VIC_AGE_GROUP','VIC_SEX','VIC_RACE',
                               'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE' ], 
                   drop_first=True)
d

,distance,STATISTICAL_MURDER_FLAG,BORO_BROOKLYN,BORO_MANHATTAN,BORO_QUEENS,BORO_STATEN ISLAND,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,...,PERP_AGE_GROUP_<18,PERP_AGE_GROUP_UNKNOWN,PERP_SEX_M,PERP_SEX_U,PERP_RACE_ASIAN / PACIFIC ISLANDER,PERP_RACE_BLACK,PERP_RACE_BLACK HISPANIC,PERP_RACE_UNKNOWN,PERP_RACE_WHITE,PERP_RACE_WHITE HISPANIC
0,0.029070,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,0.005830,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0.007743,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0.002551,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.066480,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17314,0.002443,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
17315,0.003759,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
17316,0.013770,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
17317,0.013902,1,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [4]:
Xtrain_dist = d.drop(columns=['STATISTICAL_MURDER_FLAG']) #логистическая регрессия с расстоянием до ближайшей больницы среди признаков
ytrain_dist = d[['STATISTICAL_MURDER_FLAG']]

log_reg_dist = sm.Logit(ytrain_dist, Xtrain_dist).fit()
params_dist = log_reg_dist.params
params_dist=params_dist.values
params_dist

Optimization terminated successfully.
         Current function value: 0.466818
         Iterations 8


array([-0.80558557, -0.17418561, -0.21446363, -0.16235654, -0.23467541,
        0.10906068,  0.12763011,  0.45803478, -0.25241524, -0.00458885,
       -0.06395656, -0.08065024, -0.04648874, -0.18196222, -0.40447666,
       -0.912033  , -0.11211642, -0.1107518 ,  0.26623988,  0.58296294,
        0.67474783, -0.12846108, -2.58012912, -0.1431073 ,  1.57286019,
       -0.44069387, -0.84190836, -0.95497173, -1.49215267, -0.28271778,
       -0.75148831])

In [5]:
Xtrain = d.drop(columns=['STATISTICAL_MURDER_FLAG','distance']) #логистическая регрессия без расстояния до ближайшей больницы среди признаков
ytrain = d[['STATISTICAL_MURDER_FLAG']]

log_reg = sm.Logit(ytrain, Xtrain).fit()
params = log_reg.params
params=params.values
params

Optimization terminated successfully.
         Current function value: 0.466826
         Iterations 8


array([-0.17255971, -0.20988254, -0.17443986, -0.24023872,  0.1092005 ,
        0.12760464,  0.45764749, -0.2522736 , -0.00309623, -0.0641779 ,
       -0.08668986, -0.05062517, -0.18249069, -0.40517629, -0.91379249,
       -0.11782023, -0.11149119,  0.26642207,  0.58388053,  0.67354455,
       -0.12851504, -2.57881252, -0.14302271,  1.57148022, -0.45190085,
       -0.85222034, -0.96491052, -1.50205107, -0.29708973, -0.76173871])

In [6]:
probabilities_dist = expit(np.dot(Xtrain_dist, params_dist))
#логистическая функция правдоподобия для модели с расстоянием
log_likelihood_dist = np.sum(np.array(ytrain_dist) * np.log(probabilities_dist) + (1 - np.array(ytrain_dist)) * np.log(1 - probabilities_dist))

In [7]:
probabilities = expit(np.dot(Xtrain, params))
#логистическая функция правдоподобия для модели без расстояния
log_likelihood = np.sum(np.array(ytrain) * np.log(probabilities) + (1 - np.array(ytrain)) * np.log(1 - probabilities))

In [8]:
lr_stat=2*(log_likelihood-log_likelihood_dist) #LR-статистика
lr_stat

7536.656471192837

In [9]:
#проверка гипотезы с помощью хи-квадрата
df = 1
p_value = 1 - sts.chi2.cdf(lr_stat, df)

print("LLR statistic:", lr_stat)
print("Degrees of freedom:", df)
print("p-value:", p_value)

LLR statistic: 7536.656471192837
Degrees of freedom: 1
p-value: 0.0


__ВЫВОД: p-value < 0.05, основная гипотеза отвергается на 5% уровне значимости, значит, мы не можем отрицать независимость между расстоянием до ближайшей больницы и исходом шутинга (смертностью жертв).__

### 2. Проверим гипотезу о независимости между расой шутера и расой жертвы с помощью хи-квадрат критерия

In [10]:
dataa.head()

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,Latitude,Longitude,Lon_Lat,distance
0,261190187,2022-12-31,23:41:00,QUEENS,105,0,18-24,M,BLACK,<18,M,BLACK,40.693091,-73.733598,POINT (-73.733598 40.693091),0.029070
1,261175635,2022-12-30,16:26:00,BRONX,44,0,18-24,M,WHITE HISPANIC,45-64,M,BLACK,40.837369,-73.906691,POINT (-73.906691 40.837369),0.005830
2,261176929,2022-12-30,17:00:00,MANHATTAN,30,0,25-44,M,BLACK,25-44,M,BLACK,40.825160,-73.949519,POINT (-73.949519 40.82516),0.007743
3,261120108,2022-12-29,16:48:00,BRONX,42,1,18-24,F,BLACK,25-44,M,BLACK,40.830025,-73.908341,POINT (-73.908341 40.830025),0.002551
4,261120017,2022-12-29,22:08:00,QUEENS,101,0,18-24,M,BLACK,25-44,F,BLACK,40.603766,-73.759286,POINT (-73.759286 40.603766),0.066480


In [11]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(dataa['VIC_RACE'], dataa['PERP_RACE'])

chi2, p_value, _, _ = chi2_contingency(contingency_table)


print("Хи-квадрат статистика:", chi2)
print("p-значение:", p_value)

Хи-квадрат статистика: 6418.518364316541
p-значение: 0.0


#### p-value < alpha = 0.05   ==>   Гипотеза о независимости расы жертвы и расы шутера отвергается на 5% уровне значимости ==> зависимость может существовать

In [12]:
contingency_table = pd.crosstab(dataa['BORO'], dataa['PERP_RACE'])

chi2, p_value, _, _ = chi2_contingency(contingency_table)


print("Хи-квадрат статистика:", chi2)
print("p-значение:", p_value)

Хи-квадрат статистика: 1332.0022874653944
p-значение: 1.673667406508859e-266


#### p-value < alpha = 0.05   ==>   Гипотеза о независимости района стрельбы и расы шутера отвергается ==> зависимость может существовать

In [13]:
contingency_table = pd.crosstab(dataa['BORO'], dataa['VIC_RACE'])

chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Хи-квадрат статистика:", chi2)
print("p-значение:", p_value)

Хи-квадрат статистика: 1438.6155665558047
p-значение: 2.7554264359815726e-289


#### p-value < alpha = 0.05   ==>   Гипотеза о независимости района стрельбы и расы жертвы отвергается ==> зависимость может существовтать


Проверим гипотезу о том, что истиная доля инцидентов с жертвой и шутером одинаковой расы более 60%

In [14]:
contingency_table = pd.crosstab(dataa['PERP_RACE'], dataa['VIC_RACE'])

successes = contingency_table.values.diagonal()
observations = contingency_table.sum(axis=1)

print(successes)
print(observations)

p = successes.sum()/observations.sum()
print(p)

[   0   52 9054  344    6  157 1000]
PERP_RACE
AMERICAN INDIAN/ALASKAN NATIVE        2
ASIAN / PACIFIC ISLANDER            154
BLACK                             11427
BLACK HISPANIC                     1314
UNKNOWN                            1802
WHITE                               283
WHITE HISPANIC                     2337
dtype: int64
0.6127951960274842


In [15]:
#H0: p = 0.6
#H1: p < 0.6
z_stat = (p - 0.6)/np.sqrt((1-p)*p/observations.sum())

z_crit = -1.65  #alpha = 0.05 односторонняя гипотеза
z_stat

3.4568481732913003

#### z_stat> z_crit => H0 не отвергается на 5% уровне значимости ==> мы не отвергаем, что вероятность того, что в инциденте стрельбы у жертвы и шутера будут одинаковые расы будет более 60%